# Clasificador Naive Bayes

<hr>

En el clasificador Naive Bayes, podemos asumir que los atributos se distribuyen normalmente.


In [12]:
import random
random.seed(42) # define the seed (important to reproduce the results)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#data = pd.read_csv('data/vertebralcolumn-3C.csv', header=(0))
data = pd.read_csv('data/Iris.csv', header=(0))

data = data.dropna(axis='rows') #remove NaN
# almacenar los nombres de las clases
classes = np.array(pd.unique(data[data.columns[-1]]), dtype=str)  

print("Número de filas y columnas de la matriz de atributos:", data.shape)
attributes = list(data.columns)
data.head(10)

Número de filas y columnas de la matriz de atributos: (150, 5)


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
5,5.4,3.9,1.7,0.4,setosa
6,4.6,3.4,1.4,0.3,setosa
7,5.0,3.4,1.5,0.2,setosa
8,4.4,2.9,1.4,0.2,setosa
9,4.9,3.1,1.5,0.1,setosa


In [13]:
data = data.to_numpy()
nrow,ncol = data.shape
y = data[:,-1]
X = data[:,0:ncol-1]

Selección de los conjuntos de entrenamiento y prueba.

In [14]:
from sklearn.model_selection import train_test_split
p = 0.7 # fracao de elementos no conjunto de treinamento
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = p, random_state = 42)

### Clasificación: aplicación del método

Inicialmente, definimos una función para calcular la densidad de probabilidad conjunta: $$p(\vec{x}|C_i) = \prod_{j=1}^d p(x_j|C_i), \quad i=1,\ldots, k$$ 
donde $C_i$ son las clases. Si la distribución es normal, tenemos que cada atributo $X_j$ tiene la siguiente función de densidad de probabilidad asociada, para cada clase:
$$
p(x_j|C_i) = \frac{1}{\sqrt{2\pi\sigma_{C_i}}}\exp \left[ -\frac{1}{2}\left( \frac{x_j-\mu_{C_i}}{\sigma_{C_i}}\right)^2 \right], \quad i=1,2,\ldots, k.
$$
Así, definimos una función para calcular la función de probabilidad.

In [15]:
def likelyhood(y, Z):
    def gaussian(x, mu, sig):
        return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))
    prob = 1
    for j in np.arange(0, Z.shape[1]):
        m = np.mean(Z[:,j])
        s = np.std(Z[:,j])      
        prob = prob*gaussian(y[j], m, s)
    return prob

A continuación, realizamos la estimación para cada clase:

In [16]:
P = pd.DataFrame(data=np.zeros((X_test.shape[0], len(classes))), columns = classes) 
for i in np.arange(0, len(classes)):
    elements = tuple(np.where(y_train == classes[i]))
    Z = X_train[elements,:][0]
    for j in np.arange(0,X_test.shape[0]):
        x = X_test[j,:]
        pj = likelyhood(x,Z)
        P[classes[i]][j] = pj*len(elements)/X_train.shape[0]

Para las observaciones del conjunto de pruebas, la probabilidad de pertenecer a cada clase:

In [17]:
P.head(10)

,setosa,versicolor,virginica
0,1.824344e-90,4.440479e-03,4.107993e-05
1,1.652256e-04,1.196823e-16,4.171552e-23
2,6.741862e-287,3.100666e-17,2.363765e-05
3,1.609452e-93,4.042876e-03,2.146958e-04
4,2.453031e-106,8.057427e-04,3.352704e-04
5,1.491009e-03,5.335375e-15,1.159063e-22
6,1.585589e-53,3.230445e-03,2.400042e-07
7,5.666865e-172,6.868359e-10,3.319537e-03
8,3.375351e-96,8.399067e-04,1.117962e-05
9,7.866680e-60,6.780257e-03,6.579053e-07


In [18]:
from sklearn.metrics import accuracy_score

y_pred = []
for i in np.arange(0, P.shape[0]):
    c = np.argmax(np.array(P.iloc[[i]]))
    y_pred.append(P.columns[c])
y_pred = np.array(y_pred, dtype=str)

score = accuracy_score(y_pred, y_test)
print('Accuracy:', score)

Accuracy: 0.9555555555555556


### Clasificación: usando la biblioteca scikit-learn

Podemos realizar la clasificación utilizando la función disponible en la biblioteca scikit-learn.

In [19]:
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

model = GaussianNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
score = accuracy_score(y_pred, y_test)
print('Accuracy:', score)

Accuracy: 0.9777777777777777


Otra forma de realizar la clasificación es suponer que los atributos tienen una distribución diferente a la normal.

Una posibilidad es suponer que los datos tienen una distribución Bernoulli. 

In [20]:
from sklearn.naive_bayes import BernoulliNB

model = BernoulliNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
score = accuracy_score(y_pred, y_test)
print('Accuracy:', score)

Accuracy: 0.28888888888888886


Código completo.

In [21]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

random.seed(42) 

data = pd.read_csv('data/Iris.csv', header=(0))

classes = np.array(pd.unique(data[data.columns[-1]]), dtype=str)  

# Convertir a matriz y vector numpy
data = data.to_numpy()
nrow,ncol = data.shape
y = data[:,-1]
X = data[:,0:ncol-1]

# Transforma los datos para que tengan media igual a cero y varianza igual a 1
#scaler = StandardScaler().fit(X) ---> no activas porque la biblioteca de NB lo hace por defecto
#X = scaler.transform(X) --> no activas porque la biblioteca de NB lo hace por defecto

# Selecciona los conjuntos de entrenamiento y prueba
p = 0.8 # fracción de elementos en el conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size = p, random_state = 42)

# ajusta el clasificador Naive-Bayes según los datos
model = GaussianNB()
model.fit(X_train, y_train)
# hace la predicción
y_pred = model.predict(X_test)
# Calcula la precisión
score = accuracy_score(y_pred, y_test)
print('Acuracia:', score)

Acuracia: 1.0


## Código completo

In [22]:
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

random.seed(42) 
data = pd.read_csv('data/Iris.csv', header=(0))
# classes: setosa, virginica e versicolor
classes = pd.unique(data[data.columns[-1]])
classes = np.array(classes, dtype=str)  
# convertir a matrices de numpy
data = data.to_numpy()
nrow,ncol = data.shape
y = data[:,-1]
X = data[:,0:ncol-1]
# Seleccionar el conjunto de prueba y de entrenamiento
p = 0.7 
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = p)

# funcao para calcular a verossimilhanca
def likelyhood(y, Z):
    def gaussian(x, mu, sig):
        return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))
    prob = 1
    for j in np.arange(0, Z.shape[1]):
        m = np.mean(Z[:,j])
        s = np.std(Z[:,j])      
        prob = prob*gaussian(y[j], m, s)
    return prob

# matriz que almacena el producto de la verosimilitud por la priori
P = pd.DataFrame(data=np.zeros((X_test.shape[0], len(classes))), columns = classes) 
for i in np.arange(0, len(classes)):
    elements = tuple(np.where(y_train == classes[i]))
    Z = X_train[elements,:][0]
    for j in np.arange(0,X_test.shape[0]):
        x = X_test[j,:]
        pj = likelyhood(x,Z) #verosimilitud
        pc = len(elements)/X_train.shape[0] # priori
        P[classes[i]][j] = pj*pc
        
# realiza la clasificación siguiendo la regla de Bayes
y_pred = []
for i in np.arange(0, P.shape[0]):
    c = np.argmax(np.array(P.iloc[[i]]))
    y_pred.append(P.columns[c])
y_pred = np.array(y_pred, dtype=str)
# Calcula la precisión en la clasificación
score = accuracy_score(y_pred, y_test)
print('Accuracy:', score)

Accuracy: 0.9333333333333333
